In [5]:
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.edmundson import EdmundsonSummarizer
from sumy.summarizers.lsa import LsaSummarizer
from sumy.summarizers.kl import KLSummarizer
from sumy.summarizers.lex_rank import LexRankSummarizer
import requests
from bs4 import BeautifulSoup
import pandas as pd
import tarfile
import io
import json
from rouge import Rouge
import numpy as np

In [6]:
def summarize_text(text, num_sentences=3):
    parser = PlaintextParser.from_string(text, Tokenizer("english"))
    
    summarizers = {
        "Edmundson": EdmundsonSummarizer(),
        "LSA": LsaSummarizer(),
        "KL": KLSummarizer(),
        "LexRank": LexRankSummarizer()
    }
    
    results = {}
    
    for name, summarizer in summarizers.items():
        if name == "Edmundson":
            summarizer.bonus_words = ["NLP", "language", "algorithms", "AI", "natural"]
            summarizer.stigma_words = ["is", "the", "a", "of", "and", "to", "in"]
            summarizer.null_words = ["and", "or", "but", "if", "then", "with", "so", "on"]
        
        summary = summarizer(parser.document, num_sentences)
        results[name] = "\n".join(str(sentence) for sentence in summary)
    
    return results

# Function to summarize from a file
def summarize_from_file(file_path, num_sentences=3):
    with open(file_path, 'r') as file:
        text = file.read()
    return summarize_text(text, num_sentences)

def summarize_from_url(url, num_sentences=3):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    text = soup.get_text()
    return summarize_text(text, num_sentences)

# Example usage
document = """
Natural Language Processing (NLP) is a field of artificial intelligence that focuses on the interaction between computers and humans using natural language. The goal is to enable machines to understand, interpret, and generate human language in a way that is meaningful and useful. NLP combines computational linguistics, rule-based modeling of human language, and machine learning to build models that can process large amounts of natural language data. The primary tasks in NLP range from basic text preprocessing to more complex tasks like language generation and understanding. Text preprocessing involves cleaning and preparing the text for analysis, breaking it into manageable units through tokenization, stemming, lemmatization, stopword removal, and part-of-speech tagging. Once text is preprocessed, syntactic and semantic analysis techniques help machines understand the structure and meaning of sentences, including parsing, dependency parsing, named entity recognition, and word sense disambiguation. Language modeling plays a key role in predicting the next word or phrase in a sentence based on the previous words, and this underpins many NLP applications like text generation, speech recognition, and machine translation. Sentiment analysis identifies the sentiment expressed in a text and is widely used in social media monitoring and customer feedback systems. Automatic text summarization reduces large pieces of text to shorter versions while retaining key information, which can be done through extractive summarization (selecting important sentences or phrases) or abstractive summarization (generating a summary in its own words). Question answering systems are designed to answer human questions based on contextual understanding of text, and they are seen in applications such as search engines and virtual assistants. NLP techniques and models have evolved significantly with the rise of deep learning. Bag of Words is a basic technique for converting text into numerical data, while TF-IDF builds on this by weighting words based on their importance in the text corpus. Word embeddings like Word2Vec, GloVe, and FastText create dense vector representations of words that capture semantic relationships between them. However, the real revolution in NLP came with the introduction of Transformer models. Transformers, like BERT and GPT, are neural networks that excel at understanding complex language patterns and are used in tasks such as text generation, sentiment analysis, and machine translation. Recurrent neural networks (RNNs) and Long Short-Term Memory (LSTM) models were previously dominant for sequence-based NLP tasks but have been largely replaced by Transformer architectures due to their superior ability to capture long-range dependencies in text. NLP is used in a wide variety of applications, from search engines and chatbots to healthcare, legal document analysis, and financial services. Despite these advances, NLP faces challenges such as the inherent ambiguity of human language, the diversity of languages and dialects, and the need for better understanding of context in conversations. Ethical concerns also arise, as NLP systems may perpetuate biases present in the training data. Nevertheless, NLP is transforming industries by enabling machines to interact with humans in more intuitive and effective ways.
"""

summaries = summarize_text(document)

for name, summary in summaries.items():
    print(f"\n{name} Summary:")
    print(summary)

# Example usage for file and URL
file_path = "/home/t/Desktop/koulu/nltk/proj/test.in"
file_summaries = summarize_from_file(file_path)

url = "https://en.wikipedia.org/wiki/Natural_language_processing"
url_summaries = summarize_from_url(url)

print("\nFile Summaries:")
for name, summary in file_summaries.items():
    print(f"\n{name} Summary:")
    print(summary)

print("\nURL Summaries:")
for name, summary in url_summaries.items():
    print(f"\n{name} Summary:")
    print(summary)


Edmundson Summary:
Natural Language Processing (NLP) is a field of artificial intelligence that focuses on the interaction between computers and humans using natural language.
NLP combines computational linguistics, rule-based modeling of human language, and machine learning to build models that can process large amounts of natural language data.
Ethical concerns also arise, as NLP systems may perpetuate biases present in the training data.

LSA Summary:
Word embeddings like Word2Vec, GloVe, and FastText create dense vector representations of words that capture semantic relationships between them.
Ethical concerns also arise, as NLP systems may perpetuate biases present in the training data.
Nevertheless, NLP is transforming industries by enabling machines to interact with humans in more intuitive and effective ways.

KL Summary:
The goal is to enable machines to understand, interpret, and generate human language in a way that is meaningful and useful.
Language modeling plays a key ro

In [8]:
# Initialize ROUGE
rouge = Rouge()

# Initialize summarizers
edmundson = EdmundsonSummarizer()
edmundson.bonus_words = ["important", "significant", "key", "central", "crucial"]
edmundson.stigma_words = ["trivial", "minor", "unimportant", "insignificant"]
edmundson.null_words = ["the", "a", "an", "in", "on", "at", "for", "of", "with"]

summarizers = {
    "Edmundson": edmundson,
    "LSA": LsaSummarizer(),
    "KL": KLSummarizer(),
    "LexRank": LexRankSummarizer()
}

def generate_summary(text, summarizer, num_sentences=3):
    parser = PlaintextParser.from_string(text, Tokenizer("english"))
    summary = summarizer(parser.document, num_sentences)
    return " ".join(str(sentence) for sentence in summary)

def evaluate_summarizer(summarizer, data):
    rouge_1_scores = []
    rouge_2_scores = []
    rouge_l_scores = []
    
    for _, row in data.iterrows():
        full_text = row['summary']
        reference_summary = row['title'] + ". " + " ".join(full_text.split()[:30])  # Use title and first 30 words as reference
        generated_summary = generate_summary(full_text, summarizer)
        
        scores = rouge.get_scores(generated_summary, reference_summary)[0]
        rouge_1_scores.append(scores['rouge-1']['f'])
        rouge_2_scores.append(scores['rouge-2']['f'])
        rouge_l_scores.append(scores['rouge-l']['f'])
    
    return np.mean(rouge_1_scores), np.mean(rouge_2_scores), np.mean(rouge_l_scores)

# Load the Wikipedia Summary Dataset
def load_wiki_summary_data(file_path, num_samples=10000):
    with tarfile.open(file_path, "r:gz") as tar:
        txt_file = [f for f in tar.getmembers() if f.name.endswith('.txt')][0]
        with tar.extractfile(txt_file) as f:
            content = io.TextIOWrapper(f, encoding='utf-8')
            data = []
            for i, line in enumerate(content):
                if i >= num_samples:
                    break
                title, summary = line.strip().split('|||')
                data.append({'title': title.strip(), 'summary': summary.strip()})
    return pd.DataFrame(data)

# Load the dataset
data = load_wiki_summary_data('raw.tar.gz', num_samples=10000)
print(f"Loaded {len(data)} samples from the dataset.")

# Evaluate each summarizer
results = {}
for name, summarizer in summarizers.items():
    print(f"Evaluating {name} summarizer...")
    rouge_1, rouge_2, rouge_l = evaluate_summarizer(summarizer, data)
    results[name] = {'ROUGE-1': rouge_1, 'ROUGE-2': rouge_2, 'ROUGE-L': rouge_l}

# Print results
for name, scores in results.items():
    print(f"{name} Summarizer:")
    print(f"  ROUGE-1: {scores['ROUGE-1']:.4f}")
    print(f"  ROUGE-2: {scores['ROUGE-2']:.4f}")
    print(f"  ROUGE-L: {scores['ROUGE-L']:.4f}")
    print()

# Analysis and comments
print("Analysis:")
print("1. Performance Comparison:")
best_rouge1 = max(results, key=lambda x: results[x]['ROUGE-1'])
best_rouge2 = max(results, key=lambda x: results[x]['ROUGE-2'])
best_rougel = max(results, key=lambda x: results[x]['ROUGE-L'])
print(f"   - Best ROUGE-1 performance: {best_rouge1}")
print(f"   - Best ROUGE-2 performance: {best_rouge2}")
print(f"   - Best ROUGE-L performance: {best_rougel}")


Loaded 100 samples from the dataset.
Evaluating Edmundson summarizer...
Evaluating LSA summarizer...
Evaluating KL summarizer...
Evaluating LexRank summarizer...
Edmundson Summarizer:
  ROUGE-1: 0.6595
  ROUGE-2: 0.6000
  ROUGE-L: 0.6587

LSA Summarizer:
  ROUGE-1: 0.4125
  ROUGE-2: 0.3041
  ROUGE-L: 0.3991

KL Summarizer:
  ROUGE-1: 0.4443
  ROUGE-2: 0.3397
  ROUGE-L: 0.4361

LexRank Summarizer:
  ROUGE-1: 0.5529
  ROUGE-2: 0.4660
  ROUGE-L: 0.5457

Analysis:
1. Performance Comparison:
   - Best ROUGE-1 performance: Edmundson
   - Best ROUGE-2 performance: Edmundson
   - Best ROUGE-L performance: Edmundson


In [20]:
from openai import OpenAI
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.edmundson import EdmundsonSummarizer
from sumy.summarizers.lsa import LsaSummarizer
from sumy.summarizers.kl import KLSummarizer
from sumy.summarizers.lex_rank import LexRankSummarizer
from rouge import Rouge
import numpy as np

# Initialize the OpenAI client
client = OpenAI(api_key='your-api-key-here')

# Initialize ROUGE
rouge = Rouge()

# Initialize summarizers
edmundson = EdmundsonSummarizer()
edmundson.bonus_words = ["NLP", "language", "algorithms", "AI", "natural"]
edmundson.stigma_words = ["is", "the", "a", "of", "and", "to", "in"]
edmundson.null_words = ["and", "or", "but", "if", "then", "with", "so", "on"]

summarizers = {
    "Edmundson": edmundson,
    "LSA": LsaSummarizer(),
    "KL": KLSummarizer(),
    "LexRank": LexRankSummarizer()
}

def generate_gpt_summary(text):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that summarizes text."},
            {"role": "user", "content": f"Summarize the following text in a few sentences: {text}"}
        ]
    )
    return response.choices[0].message.content.strip()

def generate_summary(text, summarizer, num_sentences=3):
    parser = PlaintextParser.from_string(text, Tokenizer("english"))
    summary = summarizer(parser.document, num_sentences)
    return " ".join(str(sentence) for sentence in summary)

def evaluate_summarizer(summarizer, text, gpt_summaries):
    rouge_2_scores = []
    rouge_l_scores = []
    
    generated_summary = generate_summary(text, summarizer)
    
    for gpt_summary in gpt_summaries:
        scores = rouge.get_scores(generated_summary, gpt_summary)[0]
        rouge_2_scores.append(scores['rouge-2']['f'])
        rouge_l_scores.append(scores['rouge-l']['f'])
    
    return np.mean(rouge_2_scores), np.mean(rouge_l_scores)

# Your document
document = """
Natural Language Processing (NLP) is a field of artificial intelligence that focuses on the interaction between computers and humans using natural language. The goal is to enable machines to understand, interpret, and generate human language in a way that is meaningful and useful. NLP combines computational linguistics, rule-based modeling of human language, and machine learning to build models that can process large amounts of natural language data. The primary tasks in NLP range from basic text preprocessing to more complex tasks like language generation and understanding. Text preprocessing involves cleaning and preparing the text for analysis, breaking it into manageable units through tokenization, stemming, lemmatization, stopword removal, and part-of-speech tagging. Once text is preprocessed, syntactic and semantic analysis techniques help machines understand the structure and meaning of sentences, including parsing, dependency parsing, named entity recognition, and word sense disambiguation. Language modeling plays a key role in predicting the next word or phrase in a sentence based on the previous words, and this underpins many NLP applications like text generation, speech recognition, and machine translation.
"""

# Use the GPT summaries you've already generated
gpt_summaries = [
    "Natural Language Processing (NLP) is a branch of artificial intelligence focusing on computer-human interaction through language. It aims to help machines understand, interpret, and produce human language effectively. NLP involves tasks such as text preprocessing, syntactic and semantic analysis, language modeling, sentiment analysis, text summarization, and question answering. Evolution in NLP includes the shift from traditional methods like Bag of Words and TF-IDF to advanced techniques such as Word embeddings and Transformer models like BERT and GPT. NLP has diverse applications ranging from search engines to healthcare and poses challenges like language diversity and biases in training data. Despite these challenges, NLP is revolutionizing industries by enhancing human-machine communication."
    "Natural Language Processing (NLP) is a field of artificial intelligence focused on enabling computers to understand, interpret, and generate human language. It combines computational linguistics and machine learning to process natural language data, from basic preprocessing to complex tasks like language generation and understanding. NLP tasks include text preprocessing, syntactic and semantic analysis, language modeling, sentiment analysis, text summarization, and question answering systems. Recent developments in deep learning have advanced NLP techniques, with Transformer models like BERT and GPT outperforming traditional methods like RNNs and LSTMs. NLP is widely used across industries, but challenges such as language ambiguity, diversity, and ethical concerns remain.",
    "Natural Language Processing (NLP) is a field of artificial intelligence that aims to enable computers to understand, interpret, and generate human language. NLP involves tasks like text preprocessing, syntactic and semantic analysis, language modeling, sentiment analysis, automatic text summarization, and question answering systems. Traditional NLP techniques like Bag of Words and TF-IDF have been enhanced by word embeddings and Transformer models like BERT and GPT, which excel at understanding complex language patterns. NLP is utilized in various applications such as search engines, chatbots, healthcare, legal document analysis, and financial services, but faces challenges in dealing with language ambiguity, diversity, context understanding, and ethical biases. Despite these challenges, NLP continues to transform industries by improving human-computer interactions.",
    "Natural Language Processing (NLP) is a field of artificial intelligence focused on enabling machines to understand, interpret, and generate human language effectively. It combines computational linguistics, rule-based modeling, and machine learning to process natural language data. NLP tasks include text preprocessing, syntactic and semantic analysis, language modeling, sentiment analysis, text summarization, and question answering systems. NLP techniques, such as Bag of Words, TF-IDF, and word embeddings like Word2Vec, have evolved with deep learning and Transformer models like BERT and GPT have revolutionized NLP applications. Despite challenges like language ambiguity and biases in training data, NLP is transforming industries by enhancing human-machine interaction.",
    "Natural Language Processing (NLP) is an area of artificial intelligence that focuses on computers understanding and generating human language. NLP tasks include text preprocessing, syntactic and semantic analysis, language modeling for prediction, sentiment analysis, text summarization, and question answering systems. Techniques like Bag of Words, TF-IDF, word embeddings, and Transformer models like BERT and GPT have evolved to enhance NLP capabilities. Although NLP is widely used in various applications and industries, challenges such as language ambiguity, diversity, context understanding, and ethical concerns related to biases in data remain.",
    "Natural Language Processing (NLP) is a branch of artificial intelligence that focuses on computers' interaction with human language. NLP involves tasks like text preprocessing, syntactic and semantic analysis, language modeling, sentiment analysis, text summarization, and question answering. Techniques in NLP have evolved from Bag of Words and TF-IDF to word embeddings like Word2Vec and advanced Transformer models like BERT and GPT, which excel at understanding complex language patterns. NLP is used in various applications, but faces challenges such as human language ambiguity, diversity of languages, context understanding, and ethical concerns regarding biases in training data. Despite challenges, NLP is revolutionizing industries by improving human-machine interactions.",
    "Natural Language Processing (NLP) is a field of artificial intelligence that aims to facilitate communication between computers and humans using human language. It involves tasks such as text preprocessing, syntactic and semantic analysis, language modeling, sentiment analysis, automatic text summarization, question answering systems, and more. NLP techniques have evolved, with Transformer models like BERT and GPT being widely used for tasks such as text generation and sentiment analysis. Despite challenges like language ambiguity and bias issues, NLP is making significant strides in various industries by enhancing human-computer interaction.",
    "Natural Language Processing (NLP) is a field of artificial intelligence focused on enabling computers to understand and generate human language. Techniques like text preprocessing, syntactic and semantic analysis, language modeling, sentiment analysis, automatic text summarization, and question answering systems are used to process natural language data. Recent advancements in deep learning, particularly Transformer models like BERT and GPT, have revolutionized NLP by improving language understanding and performance in various tasks. Despite challenges such as language diversity and ethical concerns around bias, NLP is reshaping industries by enhancing human-machine interaction.",
    "Natural Language Processing (NLP) is a field of artificial intelligence focused on computers interacting with humans using natural language. NLP involves tasks like text preprocessing, syntactic and semantic analysis, language modeling, sentiment analysis, text summarization, and question answering systems. Techniques in NLP have evolved from basic methods like Bag of Words to more advanced models like Transformers, such as BERT and GPT, which excel at understanding complex language patterns. NLP is used in various applications, but faces challenges like language ambiguity, diversity, and ethical concerns around biases in training data. Despite challenges, NLP is transforming industries by enabling more intuitive interactions between machines and humans.",
    "Natural Language Processing (NLP) is a field focused on computer-human interaction through human language, aiming to enable machines to understand and generate language effectively. It combines computational linguistics, rule-based modeling, and machine learning to process large amounts of text data. NLP tasks include text preprocessing, syntactic and semantic analysis, language modeling, sentiment analysis, automatic text summarization, and question answering systems. Recent advancements in deep learning, particularly Transformer models like BERT and GPT, have revolutionized NLP, replacing earlier models like RNNs and LSTMs. Despite challenges such as language diversity and ethical concerns regarding bias, NLP is widely used in various sectors like healthcare and finance, improving human-machine interactions significantly."
]

# Evaluate each summarizer
results = {}
for name, summarizer in summarizers.items():
    print(f"\nEvaluating {name} summarizer...")
    rouge_2, rouge_l = evaluate_summarizer(summarizer, document, gpt_summaries)
    results[name] = {'ROUGE-2': rouge_2, 'ROUGE-L': rouge_l}

# Print results
for name, scores in results.items():
    print(f"\n{name} Summarizer:")
    print(f"  ROUGE-2: {scores['ROUGE-2']:.4f}")
    print(f"  ROUGE-L: {scores['ROUGE-L']:.4f}")

# Analysis and comments
print("\nAnalysis:")
print("1. Performance Comparison:")
best_rouge2 = max(results, key=lambda x: results[x]['ROUGE-2'])
best_rougel = max(results, key=lambda x: results[x]['ROUGE-L'])
print(f"   - Best ROUGE-2 performance: {best_rouge2}")
print(f"   - Best ROUGE-L performance: {best_rougel}")



Evaluating Edmundson summarizer...

Evaluating LSA summarizer...

Evaluating KL summarizer...

Evaluating LexRank summarizer...

Edmundson Summarizer:
  ROUGE-2: 0.1800
  ROUGE-L: 0.3912

LSA Summarizer:
  ROUGE-2: 0.0359
  ROUGE-L: 0.2256

KL Summarizer:
  ROUGE-2: 0.1868
  ROUGE-L: 0.3711

LexRank Summarizer:
  ROUGE-2: 0.1868
  ROUGE-L: 0.3711

Analysis:
1. Performance Comparison:
   - Best ROUGE-2 performance: KL
   - Best ROUGE-L performance: Edmundson
